In [ ]:
!pip install -Uqq google-api-python-client 
!pip install -Uqq google-auth-httplib2 
!pip install -Uqq google-auth-oauthlib
!pip install -Uqq google-cloud-storage
!pip install google-cloud-translate==2.0.1

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.cloud import translate_v2

In [18]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents']

# The ID of a sample document.
DOCUMENT_ID = '17ATelHc43gJMxFfq5CS8lri1b--Vb9A4DfXx8XDcu9Q'


def main():
    """Shows basic usage of the Docs API.
    Prints the title of a sample document.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=4000)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('docs', 'v1', credentials=creds)

        # Retrieve the documents contents from the Docs service.
        document = service.documents().get(documentId=DOCUMENT_ID).execute()

        print('The title of the document is: {}'.format(document.get('title')))
        return service
    except HttpError as err:
        print(err)

def write_contents(service, text):
    requests = [
         {
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': text
            }
        },
    ]

    result = service.documents().batchUpdate(
    documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    
        
if __name__ == '__main__':
    service = main()
    
    text = "Such a nice day!"
    write_contents(service, text)
    

The title of the document is: test-docs


In [19]:
G_TRANSLATE_CREDENTIAL_PATH=""
%env GOOGLE_APPLICATION_CREDENTIALS ${G_TRANSLATE_CREDENTIAL_PATH}

In [32]:
def translate_text(target, text):
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))


In [33]:
data = translate_text('ko', "Today, I'm so thirsty")
data

Text: Today, I'm so thirsty
Translation: 오늘은 너무 목이 말라
Detected source language: en
